# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [1]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: False


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).

For your convenience, we provide the dataloader for you.


In [2]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [3]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75, 100)
        self.recurrent_layer = nn.LSTM(100, 100, 1)
        self.classify_layer = nn.Linear(100, 10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        # logits = self.project_layer(rnn_outputs[:,-1])
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [4]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

In [5]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.3120,  Accuracy: 9.50


/Users/bihuayu/Documents/MachineLearning/hw7/hw7_data/.env/lib/python2.7/site-packages/ipykernel/__main__.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


val Epoch: 0  , Loss: 2.3060,  Accuracy: 9.25


## Submit your results on Kaggle

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set.


### Testing the model and submit on Kaggle
Testing the model on the testing set and save the results as a .csv file. 
Please submitted the results.csv file generated by predict_on_test() to Kaggle(https://www.kaggle.com/t/934b80879bd741e6ac1967195604d4d9) to see how well your network performs on the test set. 
################ 3rd To Do  (20 points, the highest 3 entries get extra 10 points) ###############


In [7]:
class Net(nn.Module):
    def __init__(self, features, cls_size):
        super(Net, self).__init__()
        self.prenet = nn.Sequential(
            nn.Linear(features, 2 * hidden_size),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(2 * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout()
        )
        self.rnn1 = nn.GRU(input_size=hidden_size,
                            hidden_size=hidden_size,
                            num_layers=1, batch_first = True)
        self.dense1 = nn.Linear(hidden_size, cls_size)

    
    def forward(self, input, hidden=None):
        out_att = self.prenet(input)
        x, hidden = self.rnn1(out_att, hidden)
        logits = nn.functional.log_softmax(self.dense1(x[:,-1]))
        return logits
    
    def init_hidden(self):
        weight = next(self.parameters()).data
        return Variable(weight.new(1, batch_size, hidden_size).zero_())

features = 75
n_c = 10
hidden_size = 120
model = Net(features=features, cls_size=n_c)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
# run the model for one epoch
# can be used for both training or validation model
def train(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()


    model.train()
    hidden = model.init_hidden()
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        X_batch = Variable(sequence).type(FloatTensor)
        y_batch = Variable(label).type(LongTensor)
        #model.zero_grad()
        #output, hidden = model(X_batch, hidden.data)
        output = model(X_batch)
        loss = criterion(output, y_batch)
        #loss.backward()
        #optimizer.step()
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output.data, y_batch.data)
        confusion_matrix.add(output.data, y_batch.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 41
evaluate_every_epoch = 5
for e in range(num_epochs):
    train(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        train(valLD, model, criterion, e, False, None) 
        

#nb_epochs = 1
#for epoch in range(nb_epochs):
    #train(trLD) 
    


train Epoch: 0  , Loss: 2.3206,  Accuracy: 11.10


/Users/bihuayu/Documents/MachineLearning/hw7/hw7_data/.env/lib/python2.7/site-packages/ipykernel/__main__.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/bihuayu/Documents/MachineLearning/hw7/hw7_data/.env/lib/python2.7/site-packages/ipykernel/__main__.py:85: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


val Epoch: 0  , Loss: 2.1295,  Accuracy: 20.50
train Epoch: 1  , Loss: 1.8823,  Accuracy: 26.70
train Epoch: 2  , Loss: 1.7082,  Accuracy: 34.75
train Epoch: 3  , Loss: 1.5910,  Accuracy: 39.00
train Epoch: 4  , Loss: 1.4073,  Accuracy: 49.70
train Epoch: 5  , Loss: 1.3374,  Accuracy: 51.55
val Epoch: 5  , Loss: 1.3043,  Accuracy: 57.50
train Epoch: 6  , Loss: 1.2187,  Accuracy: 56.30
train Epoch: 7  , Loss: 1.2031,  Accuracy: 56.35
train Epoch: 8  , Loss: 1.1687,  Accuracy: 57.20
train Epoch: 9  , Loss: 1.1338,  Accuracy: 59.75
train Epoch: 10  , Loss: 1.0775,  Accuracy: 60.95
val Epoch: 10  , Loss: 1.2145,  Accuracy: 53.50
train Epoch: 11  , Loss: 1.0462,  Accuracy: 62.00
train Epoch: 12  , Loss: 1.1015,  Accuracy: 60.60
train Epoch: 13  , Loss: 1.0717,  Accuracy: 60.95
train Epoch: 14  , Loss: 1.0286,  Accuracy: 61.55
train Epoch: 15  , Loss: 1.0016,  Accuracy: 64.25
val Epoch: 15  , Loss: 1.0748,  Accuracy: 65.25
train Epoch: 16  , Loss: 1.0516,  Accuracy: 62.70
train Epoch: 17  , 

In [8]:
# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i].item())+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)
print(count)

1000


/Users/bihuayu/Documents/MachineLearning/hw7/hw7_data/.env/lib/python2.7/site-packages/ipykernel/__main__.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Report the performance
################ 4th To Do  (5 points)##################
network architecture:
I have two layers of a combination of a linear layer + ReLU + Dropput. Then, a GRU is used. Then, a log_softmax is used.
optimiziter: Adam
learning rate: 0.01
epochs: 52

Report your Kaggle Performance here: 0.52200 rank 68